In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("phucthaiv02/butterfly-image-classification")

print("Path to dataset files:", path)

100%|██████████| 226M/226M [00:59<00:00, 3.96MB/s] 

Extracting files...


Path to dataset files: /Users/rujutabhanose/.cache/kagglehub/datasets/phucthaiv02/butterfly-image-classification/versions/3


In [10]:
base_path = "/Users/rujutabhanose/.cache/kagglehub/datasets/phucthaiv02/butterfly-image-classification/versions/3"
train_dir = os.path.join(base_path, "train")
test_dir = os.path.join(base_path, "test")

train_df = pd.read_csv(os.path.join(base_path, "Training_set.csv"))
test_df = pd.read_csv(os.path.join(base_path, "Testing_set.csv"))

print("Training set:")
print(train_df.head())

Training set:
      filename                     label
0  Image_1.jpg          SOUTHERN DOGFACE
1  Image_2.jpg                    ADONIS
2  Image_3.jpg            BROWN SIPROETA
3  Image_4.jpg                   MONARCH
4  Image_5.jpg  GREEN CELLED CATTLEHEART


In [11]:
#Preprocess data
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])

print("Unique classes:", label_encoder.classes_)

# Ensure label column is string type
train_df["label"] = train_df["label"].astype(str)

Unique classes: ['ADONIS' 'AFRICAN GIANT SWALLOWTAIL' 'AMERICAN SNOOT' 'AN 88' 'APPOLLO'
 'ATALA' 'BANDED ORANGE HELICONIAN' 'BANDED PEACOCK' 'BECKERS WHITE'
 'BLACK HAIRSTREAK' 'BLUE MORPHO' 'BLUE SPOTTED CROW' 'BROWN SIPROETA'
 'CABBAGE WHITE' 'CAIRNS BIRDWING' 'CHECQUERED SKIPPER' 'CHESTNUT'
 'CLEOPATRA' 'CLODIUS PARNASSIAN' 'CLOUDED SULPHUR' 'COMMON BANDED AWL'
 'COMMON WOOD-NYMPH' 'COPPER TAIL' 'CRECENT' 'CRIMSON PATCH'
 'DANAID EGGFLY' 'EASTERN COMA' 'EASTERN DAPPLE WHITE'
 'EASTERN PINE ELFIN' 'ELBOWED PIERROT' 'GOLD BANDED' 'GREAT EGGFLY'
 'GREAT JAY' 'GREEN CELLED CATTLEHEART' 'GREY HAIRSTREAK' 'INDRA SWALLOW'
 'IPHICLUS SISTER' 'JULIA' 'LARGE MARBLE' 'MALACHITE' 'MANGROVE SKIPPER'
 'MESTRA' 'METALMARK' 'MILBERTS TORTOISESHELL' 'MONARCH' 'MOURNING CLOAK'
 'ORANGE OAKLEAF' 'ORANGE TIP' 'ORCHARD SWALLOW' 'PAINTED LADY'
 'PAPER KITE' 'PEACOCK' 'PINE WHITE' 'PIPEVINE SWALLOW' 'POPINJAY'
 'PURPLE HAIRSTREAK' 'PURPLISH COPPER' 'QUESTION MARK' 'RED ADMIRAL'
 'RED CRACKER' 'RED POSTMA

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Convert labels to strings (important for sparse mode)
train_df["label"] = train_df["label"].astype(str)

# Create ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Image size & batch
img_size = (128, 128)
batch_size = 32

# Create training and validation generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col="filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="sparse",
    subset="training",
    shuffle=True,
    seed=42
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col="filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="sparse",
    subset="validation",
    shuffle=True,
    seed=42
)


Found 5200 validated image filenames belonging to 75 classes.
Found 1299 validated image filenames belonging to 75 classes.


In [13]:
#build CNN
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 75)             │         9,675 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,314,315 (12.64 MB)

 Trainable params: 3,314,315 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Epoch 1/10


/Users/rujutabhanose/Documents/Deep Learning/.venv/lib/python3.13/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 73s 442ms/step - accuracy: 0.0581 - loss: 4.0400 - val_accuracy: 0.1778 - val_loss: 3.3797
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 61s 375ms/step - accuracy: 0.1994 - loss: 3.1849 - val_accuracy: 0.3641 - val_loss: 2.4710
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 61s 377ms/step - accuracy: 0.3446 - loss: 2.4950 - val_accuracy: 0.4804 - val_loss: 2.0695
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 880s 5s/step - accuracy: 0.4523 - loss: 1.9976 - val_accuracy: 0.5173 - val_loss: 1.7906
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 61s 373ms/step - accuracy: 0.5458 - loss: 1.6180 - val_accuracy: 0.5612 - val_loss: 1.6903
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 87s 535ms/step - accuracy: 0.6179 - loss: 1.3027 - val_accuracy: 0.5566 - val_loss: 1.6505
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 70s 427ms/step - accuracy: 0.6792 - loss: 1.0829 - val_accuracy: 0.5643 - val_loss: 1.6711
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 63s 388ms/step - accuracy: 0.7313 - loss: 0.8977 - val_a

In [16]:
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.6028 - loss: 1.6476
Validation Loss: 1.6476
Validation Accuracy: 0.6028


In [17]:
df_test = pd.read_csv(os.path.join(base_path, "Testing_set.csv"))
df_test["filename"] = df_test["filename"].astype(str)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=test_dir,
    x_col="filename",
    y_col=None,
    target_size=img_size,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

Found 2786 validated image filenames.


In [18]:
import numpy as np

preds = model.predict(test_generator)
predicted_class_indices = np.argmax(preds, axis=1)

# Map class indices back to labels
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predicted_labels = [labels[k] for k in predicted_class_indices]

# Save predictions
df_test["predicted_label"] = predicted_labels
df_test.to_csv("test_predictions.csv", index=False)

print("Predictions saved to test_predictions.csv")

   1/2786 ━━━━━━━━━━━━━━━━━━━━ 4:15 92ms/step

/Users/rujutabhanose/Documents/Deep Learning/.venv/lib/python3.13/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2786/2786 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step
Predictions saved to test_predictions.csv
